# Implementacion del algoritmo CLR

> Recomendaciones
* Para las variables binarias no se requiere hacer codificacion one - hot
* Probar con el regresor ElasticNet y verificar si efectivanete elimina variables colineales en cada grupo.
* Investigar un regresor que elimine la colinealidad entre variables.
* Modificar el concepto de error promedio aceptado.
* Modificar el numero minimo de regristros por grupo, al menor el 10% de los datos, registros minimos.
* Seleccionar un error promedio aceptado Correcto.

In [26]:
# Dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoCV
from numpy import mean
from numpy import std
from numpy import arange
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [27]:
# Read to dataset
df = pd.read_excel("../Archivos Generados/DatasetFinal.xlsx")
# Ornial variables list
bin_features =['SEM_TRATADAS','DRENAJE','ALMACENAMIENTO_FINCA','CAP_ENDURE_RASTA','MOTEADOS_RASTA','MOTEADOS_MAS70cm._RASTA',
               'OBSERVA_EROSION_RASTA','OBSERVA_MOHO_RASTA','OBSERVA_RAICES_VIVAS_RASTA','OBSERVA_HOJARASCA_MO_RASTA',
                'SUELO_NEGRO_BLANDO_RASTA','CUCHILLO_PRIMER_HTE_RASTA','CERCA_RIOS_QUEBRADAS_RASTA',
               ]



'''
    Esta clase permite realizar la codificación en caliente especificameente variables categoricas
'''

class OneHotCoding():
    def __init__(self, df, bin_features):
        self.bin_features = bin_features
        self.df = df
    
    # Metodo para realizar la codificacion dummy a las variables categoricas

    def dummyCodification(self):
        cat_features = self.df.select_dtypes(include = ["object", "category"]).columns
        bin_dataset = self.df[self.bin_features].replace({'SI': 1, 'NO': 0})
        categorical_features = [x for x in cat_features if x not in self.bin_features]
        df_cat = pd.get_dummies(df[categorical_features])
        self.df.drop(cat_features, axis = 1, inplace = True)
        df_final = pd.concat([self.df,df_cat,bin_dataset ], axis = 1)
        df_final.to_excel("../Archivos Generados/PipelineResults/dasetOneHot.xlsx")
        print("Ejeción Terminada")
        return df_final


#categorical_transformer = Pipeline(
#    steps=[("OneHotCoding",  OneHotCoding(df,bin_features).dummyCodification())]
#)


class LinearRegession():
    def __init__(self, df, alpha, l1_ratio):
        self.df = df
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    

    def CalcularModeloLR(self):
        # alpha=0.1, l1_ratio=0.97
        Y = self.df.RDT_AJUSTADO
        X = self.df.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1) 
        modelElasticNet = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio, random_state=123)
        model = modelElasticNet.fit(X,Y)
        r_2 = model.score(X,Y)
        # Pedicciones
        yhat = model.predict(X)

        return [model, r_2, yhat]


class CLR():

    def __init__(self, df, yhat):
        self.df = df
        self.yhat = yhat

    
    def calcularMAE(self):
        EPA = 0
        Acumulador = 0
        accepted_average_error= []
        self.df["yhat"]= pd.Series(yhat)
        self.df["EA"] = abs(self.df.RDT_AJUSTADO - self.df.yhat)
        self.df_ordely = self.df.sort_values(by=['EA'],ascending=True).reset_index()

        # We calculated MAE
        for i in range(len(self.df_ordely)):
            Acumulador = Acumulador + self.df_ordely.loc[i].EA
            EPA = Acumulador/(i+1)
            accepted_average_error.append(EPA)
        
        # we agregate the average to dataset ordely
        self.df_ordely["MAE"] = pd.Series(accepted_average_error)
        #self.df_ordely.to_excel("../Archivos Generados/PipelineResults/DatasetOrdenadoMAE.xlsx")
        #self.df.to_excel("../Archivos Generados/PipelineResults/DatasetOriginal.xlsx")
        return self.df_ordely
    


def DeleteRecordsGroup(Group, df):
    indexEliminar = list(Group["index"])
    df_new = df[df.index.isin(indexEliminar)== False]
    return df_new



def compareCorrelation(CorelacionGruposCalidad, nuevaCorrelation, listaGrupos):
    lista_grupos = listaGrupos
    print("Corrlacion original: ", CorelacionGruposCalidad)
    print("Nueva Corrlacion: ", nuevaCorrelation)
    print("Longitud de la lista de grupos: ", len(lista_grupos))
    arr =  np.array(CorelacionGruposCalidad) - np.array(nuevaCorrelation)
    # Posicion del grupo amantener
    position = np.where(arr == np.amin(arr))
    print("====================ARRAY ======================", arr)
    print("=================== POSITION ===================", position)
    # Lista de grupos eliminar registros
    indexGrupoModificar = position[0][0]
    lista_grupos.pop(indexGrupoModificar)
    return lista_grupos, indexGrupoModificar
    

In [28]:
# Variables Globales
# =========================================================================================
Minimum_records = 80
minimum_correlation= 0.88
MAE_Allowed = 45
group_acepted = []
model_acepted = []
correlation_model = []
additional_average_error = 50
contador = 0



#  FASE 1
# =========================================================================================

dataset = OneHotCoding(df,bin_features).dummyCodification()
while (len(dataset) !=0):
    contador = contador+1
    print("Tamaño del dataset: ", dataset.shape)
    modellr, r_2, yhat = LinearRegession(dataset, 0.1, 0.97).CalcularModeloLR()
    print("Ajuste del Modelo dataset Completo: ", r_2)
    DatasetOrdely = CLR(dataset,yhat).calcularMAE()
    group = DatasetOrdely.loc[DatasetOrdely.MAE < MAE_Allowed]
    if (len(group) >= Minimum_records):
            print("El grupo cumple minimo de registros")
            group = group.drop(["yhat","EA","MAE"], axis=1)
            dataset = dataset .drop(["yhat","EA"], axis=1)
            group_model, r2_group_mode, yhat_group = LinearRegession(group,0.1,0.97).CalcularModeloLR()
            print(f"R2 del grupo {contador}: ",r2_group_mode)
            if(r2_group_mode >= 0.88):
                # Elimino los registros para la siguiente iteración
                dataset = DeleteRecordsGroup(group, dataset)
                group = group.drop(['index'], axis=1)
                group_acepted.append(group)
                model_acepted.append(group_model)
                correlation_model.append(r2_group_mode)
                group.to_excel(f"../Archivos Generados/PipelineResults/Grupo{contador}.xlsx")
                MAE_Allowed = MAE_Allowed + additional_average_error
            else:
                break  
    else:
        Orphans = dataset
        print("Logitud Huerfanos: ", Orphans.shape)
        # Aqui hay que eliminar columnas EA , yhat
        Orphans.to_excel("../Archivos Generados/PipelineResults/Huerfanos.xlsx")
        break


Ejeción Terminada
Tamaño del dataset:  (799, 176)
Ajuste del Modelo dataset Completo:  0.7420467584701328


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.906e+07, tolerance: 1.533e+05
  model = cd_fast.enet_coordinate_descent(


El grupo cumple minimo de registros
R2 del grupo 1:  0.9984019440207271


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.784e+05, tolerance: 7.284e+03
  model = cd_fast.enet_coordinate_descent(


Tamaño del dataset:  (717, 176)
Ajuste del Modelo dataset Completo:  0.7302764790023766


...

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.116e+07, tolerance: 1.355e+05
  model = cd_fast.enet_coordinate_descent(


El grupo cumple minimo de registros
R2 del grupo 3:  0.9930874180899754


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.038e+05, tolerance: 8.809e+03
  model = cd_fast.enet_coordinate_descent(


Tamaño del dataset:  (546, 176)
Ajuste del Modelo dataset Completo:  0.7586428604562754
Logitud Huerfanos:  (546, 178)


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.002e+07, tolerance: 1.266e+05
  model = cd_fast.enet_coordinate_descent(


In [29]:
Orphans = Orphans.drop(["yhat", "EA"],axis=1).reset_index(drop=True)

In [30]:
Orphans_10 = Orphans

In [31]:
# Variables de entrada
GruposCalidad = group_acepted
CorelacionGruposCalidad = correlation_model


In [7]:
CorelacionGruposCalidad

[0.9984019440207271, 0.9892407435418807, 0.9930874180899754]

In [32]:
# Longitudes iniciales de los grupos de calidad
print(f"Grupo Calidad 1: {len(GruposCalidad[0])} , Grupo de Calidad 2: {len(GruposCalidad[1])}, Grupo de Calidad 3: {len(GruposCalidad[2])}")

Grupo Calidad 1: 82 , Grupo de Calidad 2: 84, Grupo de Calidad 3: 87


In [33]:
listaGrupos = [0, 1, 2]
nuevaCorrelation= []
corr = []
for register in range(len(Orphans_10)):
    print("Registro numero: ", register)
    for group in range(len(GruposCalidad)):
        # Agreogo cada huerfano al grupo de caliad
        GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
        print("Longitud grupo de calidad: ", len(GruposCalidad[group]))
        # Se obtiene el nuevo modelo y el r2 de cada grupo
        new_mode, new_r2, new_that = LinearRegession(GruposCalidad[group],0.1, 0.97).CalcularModeloLR()
        # Agrego la nueva correlacion
        nuevaCorrelation.append(new_r2)
        

    # Comparo las correlaciones nuevas con las enteriores y obtengo los id de lso grupos  Elimina registros
    #corr.append((CorelacionGruposCalidad, nuevaCorrelation))
    lista_id_grupos_retirar, index = compareCorrelation(CorelacionGruposCalidad, nuevaCorrelation, listaGrupos)
    CorelacionGruposCalidad[index] = nuevaCorrelation[index]
    print("Grupo Modificar: ", index)
    # Se limpia la lista de las correlaciones
    nuevaCorrelation = []
    print("lista de grupos: ",listaGrupos)
    listaGrupos = [0, 1, 2]
    # Elimino los registros
    for i in lista_id_grupos_retirar:
        GruposCalidad[i].drop([len(GruposCalidad[i]) -1 ],axis=0, inplace=True)

    #Eliminar ese registro en el dataset de huerfanos
    Orphans_10.drop([0],axis=0, inplace=True)
    Orphans_10 = Orphans_10.reset_index(drop=True)
    

Registro numero:  0
Longitud grupo de calidad:  83
Longitud grupo de calidad:  85
Longitud grupo de calidad:  88
Corrlacion original:  [0.9984019440207271, 0.9892407435418807, 0.9930874180899754]
Nueva Corrlacion:  [0.9983731059668111, 0.9823222011296523, 0.9922414694217224]
Longitud de la lista de grupos:  3
====================ARRAY ====================== [2.88380539e-05 6.91854241e-03 8.45948668e-04]
=================== POSITION =================== (array([0], dtype=int64),)
Grupo Modificar:  0
lista de grupos:  [1, 2]


<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.217e+05, tolerance: 7.365e+03
  model = cd_fast.enet_coordinate_descent(
<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:6

Registro numero:  1
Longitud grupo de calidad:  84
Longitud grupo de calidad:  85
Longitud grupo de calidad:  88
Corrlacion original:  [0.9983731059668111, 0.9892407435418807, 0.9930874180899754]
Nueva Corrlacion:  [0.9983820877581946, 0.9816209518993967, 0.9905011130674355]
Longitud de la lista de grupos:  3
====================ARRAY ====================== [-8.98179138e-06  7.61979164e-03  2.58630502e-03]
=================== POSITION =================== (array([0], dtype=int64),)
Grupo Modificar:  0
lista de grupos:  [1, 2]
Registro numero:  2
Longitud grupo de calidad:  85


<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.304e+05, tolerance: 7.370e+03
  model = cd_fast.enet_coordinate_descent(
<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:6

Longitud grupo de calidad:  85
Longitud grupo de calidad:  88
Corrlacion original:  [0.9983820877581946, 0.9892407435418807, 0.9930874180899754]
Nueva Corrlacion:  [0.9983993635574246, 0.9821724543012057, 0.9930825436389433]
Longitud de la lista de grupos:  3
====================ARRAY ====================== [-1.72757992e-05  7.06828924e-03  4.87445103e-06]
=================== POSITION =================== (array([0], dtype=int64),)
Grupo Modificar:  0
lista de grupos:  [1, 2]
Registro numero:  3
Longitud grupo de calidad:  86
Longitud grupo de calidad:  85
Longitud grupo de calidad:  88


...

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.635e+06, tolerance: 9.106e+04
  model = cd_fast.enet_coordinate_descent(
<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.156e+06, tolerance: 1.036e+04
  model = cd_fast.enet_coordinate_descen

Longitud grupo de calidad:  85
Longitud grupo de calidad:  318
Corrlacion original:  [0.8989830561459263, 0.9892407435418807, 0.9284900459457452]
Nueva Corrlacion:  [0.8990194073069048, 0.9813420536940695, 0.9280887980251674]
Longitud de la lista de grupos:  3
====================ARRAY ====================== [-3.63511610e-05  7.89868985e-03  4.01247921e-04]
=================== POSITION =================== (array([0], dtype=int64),)
Grupo Modificar:  0
lista de grupos:  [1, 2]
Registro numero:  545
Longitud grupo de calidad:  398


<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.622e+06, tolerance: 9.173e+04
  model = cd_fast.enet_coordinate_descent(
<ipython-input-33-82ad39f4a72a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  GruposCalidad[group] = GruposCalidad[group].append(Orphans_10.loc[0], ignore_index = True)
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:6

Longitud grupo de calidad:  85
Longitud grupo de calidad:  318
Corrlacion original:  [0.8990194073069048, 0.9892407435418807, 0.9284900459457452]
Nueva Corrlacion:  [0.8995441127926722, 0.9830639616250663, 0.9233252643903729]
Longitud de la lista de grupos:  3
====================ARRAY ====================== [-0.00052471  0.00617678  0.00516478]
=================== POSITION =================== (array([0], dtype=int64),)
Grupo Modificar:  0
lista de grupos:  [1, 2]


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.724e+07, tolerance: 5.193e+04
  model = cd_fast.enet_coordinate_descent(


In [14]:
# [0.9984019440207271, 0.9892407435418807, 0.9930874180899754]- [0.9983731059668111, 0.9823222011296523, 0.9922414694217224]
0.9983731059668111 - 0.9984019440207271 

-2.8838053916069306e-05

In [34]:
# Longitudes iniciales de los grupos de calidad
print(f"Grupo Calidad 1: {len(GruposCalidad[0])} , Grupo de Calidad 2: {len(GruposCalidad[1])}, Grupo de Calidad 3: {len(GruposCalidad[2])}")

Grupo Calidad 1: 398 , Grupo de Calidad 2: 84, Grupo de Calidad 3: 317


In [35]:
# Nueva correlacion al agregar el nuevo registro
CorelacionGruposCalidad

[0.8995441127926722, 0.9892407435418807, 0.9284900459457452]

In [36]:
Orphans_10

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA


In [ ]:
GruposCalidad[0]

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,2481.0,6.0,45.0,77.0,54000.0,11.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2660.0,5.0,54.0,80.0,60000.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,3018.0,4.0,48.0,82.0,55000.0,12.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2450.0,5.0,47.0,73.0,70000.0,72.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2465.0,5.0,47.0,81.0,67000.0,14.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,269.0,6.0,42.0,86.0,60000.0,9.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
624,269.0,6.0,42.0,86.0,60000.0,9.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
625,269.0,6.0,42.0,86.0,60000.0,9.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
626,269.0,6.0,42.0,86.0,60000.0,9.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
GruposCalidad[0].RDT_AJUSTADO

0      3813.95
1      4000.00
2      4958.14
3      4409.30
4      4834.88
        ...   
623    5116.28
624    5116.28
625    5116.28
626    5116.28
627    5116.28
Name: RDT_AJUSTADO, Length: 628, dtype: float64

In [ ]:
# mandamos ese grupo de calidad al reegresor
a,b,c = LinearRegession(GruposCalidad[0],0.1, 0.97).CalcularModeloLR()

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.522e+06, tolerance: 7.721e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
d,e,f = LinearRegession(GruposCalidad[1],0.1, 0.97).CalcularModeloLR()

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.131e+06, tolerance: 1.026e+04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
g,h,i = LinearRegession(GruposCalidad[2],0.1, 0.97).CalcularModeloLR()

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.296e+05, tolerance: 8.809e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
a

ElasticNet(alpha=0.1, l1_ratio=0.97)

In [ ]:
b

0.9868945270361709

In [ ]:
len(a.coef_)

174

In [ ]:
# Se gurdan los coeficientes del modelo en un excel.
# se determina las caracteristicas asociadas a cada modelo
CoefG = pd.DataFrame()
#CoefG["Columns"] = dataset.columns
CoefG['Coefficientes G1'] = pd.Series(a.coef_)
CoefG['Coefficientes G2'] = pd.Series(d.coef_)
CoefG['Coefficientes G3'] = pd.Series(g.coef_)
CoefG.to_excel("../Archivos Generados/CoeficientesGrupos.xlsx")

In [ ]:
Orphans

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA


In [ ]:
CorelacionGruposCalidad

[0.8995441127926722, 0.9892407435418807, 0.9284900459457452]

In [ ]:
#Nueva correlacion al agrega el nuevo registro
correlation_model

[0.8995441127926722, 0.9892407435418807, 0.9284900459457452]

In [ ]:
np.array(nuevaCorrelation) - np.array(correlation_model)

ValueError: operands could not be broadcast together with shapes (0,) (3,) 

In [ ]:
np.amax(np.array([-1,-2, -3, -0.00002883, -0.0069185, -0.000899]))

-2.883e-05

In [ ]:
# Resto siempree la del modelo original ---   (menos correlacion de registros agregados)
arr = np.array(nuevaCorrelation) -np.array(correlation_model)
arr

array([-2.88380539e-05, -6.91854241e-03, -8.45948668e-04])

In [ ]:
np.amax(np.array(nuevaCorrelation) -np.array(correlation_model))

-2.8838053916069306e-05

In [18]:
np.amax([ 0.00066099, 0.00119067, -0.00022128])

0.00119067

In [19]:
np.amin([ 0.00066099, 0.00119067, -0.00022128])

-0.00022128

In [25]:
np.amin([ 0.00066099, 0.00119067, 0.00006352,  -0.00022128, -0.00526, -0.52, -0.00000563])

-0.52

In [ ]:
# Obtener el grupo donde el registro genera menor| mayor  impacto
position = np.where(arr == np.amax(arr))
position[0][0]

0

In [ ]:
lista_Gruppos = [0, 1, 2, 3]
lista_Gruppos.pop(position[0][0])
lista_Gruppos

[0, 2, 3]

In [ ]:
lista_Gruppos

[0, 2, 3]

In [ ]:
Numero_Grupos = range(0, len(GruposCalidad), 1)
Numero_Grupos

range(0, 3)

In [ ]:
len(GruposCalidad[1])

86

In [ ]:
GruposCalidad[1].drop([len(GruposCalidad[1]) -1 ],axis=0, inplace=True)

In [ ]:
GruposCalidad[1]

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,3943.0,3.0,48.0,79.0,63000.0,6.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
1,469.0,5.0,50.0,70.0,60000.0,11.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
2,2908.0,5.0,46.0,95.0,73000.0,5.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,3739.0,4.0,47.0,82.0,62000.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,2490.0,6.0,40.0,88.0,60000.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,3749.0,4.0,48.0,82.0,60000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
80,2930.0,4.0,46.0,79.0,62000.0,3.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
81,3683.0,4.0,47.0,90.0,62000.0,5.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
82,3733.0,4.0,46.0,81.0,62000.0,8.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [ ]:
# Retirar los registros del grupo
lista_id_grupos_retirar_registros = [1,2]
for i in lista_id_grupos_retirar:
    GruposCalidad[i].drop([len(GruposCalidad[i]) -1 ],axis=0, inplace=True)

IndexError: index 86 is out of bounds for axis 0 with size 86

<te>